In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import tensorflow as tf

# 手写数字的图像识别，input data是28x28黑白图片，用LR建模

LR 用于计算input属于某个target class 的概率，本例中是[0-9]
![Logistic Regression classifier expression](LR_classifier_expression.png)
**我们的目标是找出最优的 W 和 b**

## Logistic Regression network
![Logistic Regression network](LR_network.png)
这个LR网络的图片解释相当简陋。它没有隐层，意味它学习复杂关系的能力有限。
input layer 有28x28=784个neurons，每个像素对应一个neuron。

## LR建模4个阶段
    1. inference: 给定一个minibatch（inputs），生成其output class 的概率分布
    2. loss: 计算 error function 的值，本例中采用cross-entropy loss
    3. training: 计算参数的梯度并更新模型（函数表达式），即W,b 的梯度下降
    4. evaluate: 评估一个模型的有效性

### 1. inference: 给定一个minibatch，生成其output class 的概率分布
给定一个minibatch，它由784维向量组成，每个向量表示一张28x28的image。
logistic regression 模型的表达式为![Logistic Regression classifier expression](LR_classifier_expression.png)
**x**是input，矩阵W表示输入和输出层之间的连接权重

In [3]:
def inference(x):
    tf.constant_initializer(value=0)
    W = tf.get_variable("W", [784, 10], initializer=init)
    b = tf.get_variable('b', [10], initializer=init)
    output = tf.nn.softmax(tf.matmul(x, W) + b)  # 输出是一个概率值P
    return output

### 2. loss: 计算 error function 的值，本例中采用cross-entropy loss
接下来，给出minibatch 的真实labels **y**，与outputs比较 以计算每个数据样本的平均误差。

cross-entropy ≠ loss function。设p是真label的概率，q是预测的概率，交叉熵度量p和q的相似度（距离）：
![cross-entropy](cross_entropy.svg)

Log loss =  logistic loss = cross-entropy loss：

![log loss](cross_entropy_loss.svg)

In [4]:
def loss(output, y):
    dot_product = y * tf.log(output)
    xentropy = -tf.reduce_sum(dot_product, reduction_indices=1) # 沿轴0缩减是每column折叠为单个值，轴1是每row；
    loss = tf.reduce_mean(xentropy)                                # 通常沿轴i 即将向量的第i维折叠为单个值。
    return loss    

### 3. training: 计算参数的梯度并更新模型（函数表达式），即W,b 的梯度下降

有了cost function之后，接下来计算梯度并优化模型的参数。注意对已迭代minibatch 进行计数的参数global_step

In [6]:
def training(cost, global_step):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate) # learning_rate 在函数外自定义
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

### 4. evaluate: 评估一个模型的有效性

最后组装一个computational subgraph 来评估模型在validation 或 test set 上的表现

In [7]:
def evaluate(output, y):
    correct_predicton = tf.equal(tf.argmax(output, 1),
                                 tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

以上就是LR模型的TensorFlow graph setup

## 训练模型并记录日志

训练模型时，为了log 重要信息，我们要将几个概要统计量（summary statistics）记入日志。

例如用***tf.scalar_summary*** 和 ***tf.histogram_summary***  to

    log the cost for each minibatch, validation error, and the distribution of parameters.

In [ ]:
#例：cost function的标量摘要统计量
def training(cost, global_step):
    tf.scalar_summary('cost',cost)  # 比上面多了这一行
    optimizer = tf.train.GrandientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

每个epoch， 我们可以用 ***tf.merge_all_summaries*** 来收藏所有已经记入日志的summary statistics，

并可用***tf.train.SummaryWriter*** 将日志写入磁盘

除了存储summary statistics，我们也可以用***tf.train.Saver***存储训练好的模型参数，以便未来调用。



# 将以上整合一下，加入参数值，就是一个python 脚本了。

一张image里一位数字的识别，用logistic regression模型能达到91.1的准确率，即8.1%的错误率。

貌似不错，但在高价值的实际应用上并不是很有用。

比如，4位数的识别，1000-9999，那么错误率几乎是30%了。因此，我们接下来用feed-forward network 模型来试试看。

# 用TensorBoard可视化computational graph

训练完成，有了summary statistics的logging 之后，我们就可以将收集的数据进行可视化。

     tensorboard --logdir=<absolute_path_to_log_dir>